In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import os
import pyspark

scala_version = '2.12'  # TODO: Ensure this is correct
spark_version = pyspark.__version__

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.5.1'
]

args = os.environ.get('PYSPARK_SUBMIT_ARGS', '')
if not args:
    args = f'--packages {",".join(packages)}'
    print('Using packages', packages)
    os.environ['PYSPARK_SUBMIT_ARGS'] = f'{args} pyspark-shell'
else:
    print(f'Found existing args: {args}') 

spark = SparkSession.builder\
   .master("local")\
   .appName("kafka-example")\
   .getOrCreate()


Using packages ['org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0', 'org.apache.kafka:kafka-clients:3.5.1']


In [3]:
spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka-1:9092,kafka-2:9092") \
  .option("subscribe", "entrada") \
  .load() \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .select(explode(split("value", " ")).alias("word")) \
  .groupBy("word").count() \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka-1:9092,kafka-2:9092") \
  .option("topic", "salida") \
  .option("checkpointLocation", "/home/jovyan/checkpoint") \
  .trigger(continuous="1 second") \
  .start()

AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;
Aggregate [word#51], [word#51, count(1) AS count#55L]
+- Project [word#51]
   +- Generate explode(split(value#47,  , -1)), false, [word#51]
      +- Project [cast(key#32 as string) AS key#46, cast(value#33 as string) AS value#47]
         +- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@df309da, kafka, org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaTable@7af3f0a5, [kafka.bootstrap.servers=kafka-1:9092,kafka-2:9092, subscribe=entrada], [key#32, value#33, topic#34, partition#35, offset#36L, timestamp#37, timestampType#38], StreamingRelation DataSource(org.apache.spark.sql.SparkSession@49dbd6da,kafka,List(),None,List(),None,Map(kafka.bootstrap.servers -> kafka-1:9092,kafka-2:9092, subscribe -> entrada),None), kafka, [key#25, value#26, topic#27, partition#28, offset#29L, timestamp#30, timestampType#31]
